In [5]:
import tkinter as tk
from tkinter import messagebox, ttk
import json
import re
from datetime import date, datetime
from docx import Document
import os

# Date string to date object conversion
def str_to_date(obj):
    if isinstance(obj, str):
        try:
            return datetime.strptime(obj, "%Y-%m-%d").date()
        except ValueError:
            return obj
    return obj

# Domain Model Classes
class MedicalHistory:
    def __init__(self, conditions: list, surgeries: list, allergies: list):
        self.conditions = conditions
        self.surgeries = surgeries
        self.allergies = allergies

    def to_dict(self):
        return {"conditions": self.conditions,
                "surgeries": self.surgeries,
                "allergies": self.allergies}

class Diagnosis:
    def __init__(self, diagnosis: str, icd_code: str):
        self.diagnosis = diagnosis
        self.icd_code = icd_code

    def to_dict(self):
        return {"diagnosis": self.diagnosis,
                "icd_code": self.icd_code}

class TreatmentPlan:
    def __init__(self, exercises: list, therapies: list, modalities: list,
                 interventions: str, home_exercise_program: str):
        self.exercises = exercises
        self.therapies = therapies
        self.modalities = modalities
        self.interventions = interventions
        self.home_exercise_program = home_exercise_program

    def to_dict(self):
        return {"exercises": self.exercises,
                "therapies": self.therapies,
                "modalities": self.modalities,
                "interventions": self.interventions,
                "home_exercise_program": self.home_exercise_program}

class Assessment:
    def __init__(self, findings: str, assessment_date: date,
                 subjective_symptoms: str, objective_findings: str,
                 history: str, treatment_notes: str = "N/A"):
        self.findings = findings
        self.assessment_date = assessment_date
        self.subjective_symptoms = subjective_symptoms
        self.objective_findings = objective_findings
        self.history = history
        self.treatment_notes = treatment_notes

    def to_dict(self):
        return {"findings": self.findings,
                "assessment_date": self.assessment_date,
                "subjective_symptoms": self.subjective_symptoms,
                "objective_findings": self.objective_findings,
                "history": self.history,
                "treatment_notes": self.treatment_notes}

class Reassessment:
    def __init__(self, reassessment_date: date,
                 progress_notes: str, objective_findings: str,
                 recommendations: str):
        self.reassessment_date = reassessment_date
        self.progress_notes = progress_notes
        self.objective_findings = objective_findings
        self.recommendations = recommendations

    def to_dict(self):
        return {"reassessment_date": self.reassessment_date,
                "progress_notes": self.progress_notes,
                "objective_findings": self.objective_findings,
                "recommendations": self.recommendations}

class Patient:
    def __init__(self, name: str, contact: str, dob: date, gender: str,
                 medical_aid: str, medical_aid_number: str,
                 occupation: str, referral_diagnosis: str,
                 physiotherapist_name: str,
                 medical_history: MedicalHistory,
                 diagnosis: Diagnosis,
                 treatment_plan: TreatmentPlan):
        self.name = name
        self.contact = contact
        self.dob = dob
        self.gender = gender
        self.medical_aid = medical_aid
        self.medical_aid_number = medical_aid_number
        self.occupation = occupation
        self.referral_diagnosis = referral_diagnosis
        self.physiotherapist_name = physiotherapist_name
        self.medical_history = medical_history
        self.diagnosis = diagnosis
        self.treatment_plan = treatment_plan
        self.assessments = []
        self.reassessments = []

    def add_assessment(self, assessment: Assessment):
        self.assessments.append(assessment)

    def add_reassessment(self, reassessment: Reassessment):
        self.reassessments.append(reassessment)

    def to_dict(self):
        return {"name": self.name,
                "contact": self.contact,
                "dob": self.dob,
                "gender": self.gender,
                "medical_aid": self.medical_aid,
                "medical_aid_number": self.medical_aid_number,
                "occupation": self.occupation,
                "referral_diagnosis": self.referral_diagnosis,
                "physiotherapist_name": self.physiotherapist_name,
                "medical_history": self.medical_history.to_dict(),
                "diagnosis": self.diagnosis.to_dict(),
                "treatment_plan": self.treatment_plan.to_dict(),
                "assessments": [a.to_dict() for a in self.assessments],
                "reassessments": [r.to_dict() for r in self.reassessments]}

    @classmethod
    def from_dict(cls, data):
        mh = MedicalHistory(**data["medical_history"])
        diag = Diagnosis(**data["diagnosis"])
        tp_data = data["treatment_plan"]
        tp = TreatmentPlan(tp_data.get("exercises", []),
                           tp_data.get("therapies", []),
                           tp_data.get("modalities", []),
                           tp_data.get("interventions", ""),
                           tp_data.get("home_exercise_program", ""))
        patient = cls(data["name"], data["contact"], data["dob"],
                      data["gender"], data["medical_aid"],
                      data["medical_aid_number"], data["occupation"],
                      data["referral_diagnosis"], data["physiotherapist_name"],
                      mh, diag, tp)
        # load assessments
        for a in data.get("assessments", []):
            patient.assessments.append(
                Assessment(a.get("findings", ""),
                           a.get("assessment_date", date.today()),
                           a.get("subjective_symptoms", ""),
                           a.get("objective_findings", ""),
                           a.get("history", ""),
                           a.get("treatment_notes", "N/A"))
            )
        # load reassessments
        for r in data.get("reassessments", []):
            patient.reassessments.append(
                Reassessment(r.get("reassessment_date", date.today()),
                             r.get("progress_notes", ""),
                             r.get("objective_findings", ""),
                             r.get("recommendations", ""))
            )
        return patient

# Main Application
class PhysioApp:
    def __init__(self, root):
        self.root = root

        
        # ─── Load ICD-10 lookup into memory ────────────────────────────
        with open("icd10.json", "r", encoding="utf-8") as f:
            # expect a dict: { "A00": "Cholera", … }
            self.icd_data = json.load(f)
        # ─────────────────────────────────────────────────────────────────

        self.root.title("Physiotherapy Patient Management System")
        self.root.geometry("800x600")
        self.patients = {}
        self.patient_counter = 1
        self.current_patient = None
        self.frames = {}
        self.patient_var = tk.StringVar()

        self.container = ttk.Frame(self.root)
        self.container.pack(fill="both", expand=True)

        self.create_main_menu()
        self.load_patients()

    def format_dob(self, event):
        s = re.sub(r"[^\d]", "", self.dob_entry.get())
        parts = []
        if len(s) >= 4:
            parts.append(s[:4])
            if len(s) >= 6:
                parts.append(s[4:6])
                parts.append(s[6:8])
            else:
                parts.append(s[4:])
        else:
            parts.append(s)
        new = '-'.join(parts)
        self.dob_entry.delete(0, tk.END)
        self.dob_entry.insert(0, new[:10])

    def validate_contact(self, event):
        s = re.sub(r"\D", "", self.contact_entry.get())[:10]
        self.contact_entry.delete(0, tk.END)
        self.contact_entry.insert(0, s)

    def _bind_mousewheel(self, canvas):
        canvas.bind_all("<MouseWheel>", lambda e: canvas.yview_scroll(int(-1*(e.delta/120)), "units"))

    def create_main_menu(self):
        self.clear_container()
        frame = ttk.Frame(self.container)
        frame.pack(fill="both", expand=True)
        tk.Label(frame, text="Physiotherapy Patient Management System", font=("Arial", 16)).pack(pady=20)
        tk.Button(frame, text="Patient Management", command=self.create_patient_frame).pack(pady=10)
        tk.Button(frame, text="Assessment & Reassessment", command=self.create_assessment_frame).pack(pady=10)
        tk.Button(frame, text="Generate Report", command=self.create_report_frame).pack(pady=10)
        tk.Button(frame, text="Exit", command=self.root.quit).pack(pady=10)

    def clear_container(self):
        for w in self.container.winfo_children():
            w.destroy()
        self.frames.clear()

    def create_patient_frame(self):
        self.clear_container()
        canvas = tk.Canvas(self.container)
        scrollbar = ttk.Scrollbar(self.container, orient="vertical", command=canvas.yview)
        scrollable = ttk.Frame(canvas)
        scrollable.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        canvas.create_window((0,0), window=scrollable, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        self.frames['patient'] = scrollable
        self._bind_mousewheel(canvas)

        fields = [
            ("Name", tk.Entry), ("Contact", tk.Entry), ("DOB (YYYY-MM-DD)", tk.Entry),
            ("Gender", tk.Entry), ("Medical Aid", tk.Entry), ("Medical Aid Number", tk.Entry),
            ("Occupation", tk.Entry), ("Referral Diagnosis", tk.Entry), ("Physiotherapist Name", tk.Entry)
        ]
        self.patient_entries = {}
        for i,(label, widget) in enumerate(fields):
            tk.Label(scrollable, text=label+":").grid(row=i, column=0, padx=5, pady=5, sticky='w')
            entry = widget(scrollable)
            entry.grid(row=i, column=1, padx=5, pady=5, sticky='ew')
            self.patient_entries[label] = entry
            if label == 'Contact':
                self.contact_entry = entry
                entry.bind('<KeyRelease>', self.validate_contact)
            if label.startswith('DOB'):
                self.dob_entry = entry
                entry.bind('<KeyRelease>', self.format_dob)

        # Medical history, diagnosis, treatment plan fields...
        # (omitted for brevity - same as before)

        tk.Button(scrollable, text="Add Patient", command=self.add_patient).grid(row=len(fields)+13, column=0, columnspan=2, pady=10)
        tk.Button(scrollable, text="Back to Menu", command=self.create_main_menu).grid(row=len(fields)+14, column=0, columnspan=2, pady=20)
        scrollable.columnconfigure(1, weight=1)

    def add_patient(self):
        name = self.patient_entries['Name'].get().strip()
        contact = self.contact_entry.get().strip()
        dob_str = self.dob_entry.get().strip()
        # Validate contact
        if not re.fullmatch(r"\d{10}", contact):
            return messagebox.showerror("Error", "Contact must be exactly 10 digits.")
        # Validate DOB
        try:
            dob = datetime.strptime(dob_str, "%Y-%m-%d").date()
        except ValueError:
            return messagebox.showerror("Error", "DOB must be in YYYY-MM-DD format.")
        if dob < date(1900,1,1) or dob > date.today():
            return messagebox.showerror("Error", "DOB must be between 1900-01-01 and today.")

        # Continue as before to gather other fields, build MedicalHistory, Diagnosis, TreatmentPlan...
        # (omitted for brevity)
        
        # On success:
        messagebox.showinfo("Success", f"Patient {name} added!")
        self.save_patients()

    # create_assessment_frame, create_report_frame, load, save methods remain largely unchanged,
    # but be sure to call self._bind_mousewheel(canvas) in each frame to enable scroll.

if __name__ == '__main__':
    root = tk.Tk()
    app = PhysioApp(root)
    root.mainloop()


AttributeError: 'PhysioApp' object has no attribute 'create_assessment_frame'

In [9]:
import tkinter as tk
from tkinter import messagebox, ttk
import json
import re
from datetime import date, datetime
from docx import Document
import os
import json
# … your other imports …

class PhysioApp:
    def __init__(self, root):
        # … existing init …
        # load ICD-10 dictionary
        with open("icd10.json", encoding="utf8") as f:
            raw = json.load(f)
        # precompute lowercase for fast matching
        self.icd_data = [
            {
              "code": e["code"],
              "description": e["description"],
              "lc_code": e["code"].lower(),
              "lc_desc": e["description"].lower()
            }
            for e in raw
        ]
        # … rest of init …
    def suggest_icd(self, query, limit=10):
        q = query.strip().lower()
        if not q:
            return []
        matches = []
        for e in self.icd_data:
            if q in e["lc_desc"] or e["lc_code"].startswith(q):
                matches.append(e)
                if len(matches) >= limit:
                    break
        return matches

# Date string to date object conversion
def str_to_date(obj):
    if isinstance(obj, str):
        try:
            return datetime.strptime(obj, "%Y-%m-%d").date()
        except ValueError:
            return obj
    return obj

# Domain Model Classes
class MedicalHistory:
    def __init__(self, conditions: list, surgeries: list, allergies: list):
        self.conditions = conditions
        self.surgeries = surgeries
        self.allergies = allergies

    def to_dict(self):
        return {"conditions": self.conditions,
                "surgeries": self.surgeries,
                "allergies": self.allergies}

class Diagnosis:
    def __init__(self, diagnosis: str, icd_code: str):
        self.diagnosis = diagnosis
        self.icd_code = icd_code

    def to_dict(self):
        return {"diagnosis": self.diagnosis,
                "icd_code": self.icd_code}

    def on_diagnosis_type(self, event):
        text = self.diagnosis_entry.get()
        matches = self.suggest_icd(text)
        lb = self.icd_suggestions
        lb.delete(0, tk.END)
        if not matches:
            lb.lower()
            return
        for e in matches:
            lb.insert(tk.END, f"{e['code']}: {e['description']}")
        lb.lift()  # bring to front

    def on_icd_select(self, event):
        sel = event.widget.curselection()
        if not sel: return
        line = event.widget.get(sel[0])
        code, desc = line.split(":", 1)
        self.icd_code_entry.delete(0, tk.END)
        self.icd_code_entry.insert(0, code.strip())
        self.diagnosis_entry.delete(0, tk.END)
        self.diagnosis_entry.insert(0, desc.strip())
        event.widget.delete(0, tk.END)
        event.widget.lower()

class TreatmentPlan:
    def __init__(self, exercises: list, therapies: list, modalities: list,
                 interventions: str, home_exercise_program: str):
        self.exercises = exercises
        self.therapies = therapies
        self.modalities = modalities
        self.interventions = interventions
        self.home_exercise_program = home_exercise_program

    def to_dict(self):
        return {"exercises": self.exercises,
                "therapies": self.therapies,
                "modalities": self.modalities,
                "interventions": self.interventions,
                "home_exercise_program": self.home_exercise_program}

class Assessment:
    def __init__(self, findings: str, assessment_date: date,
                 subjective_symptoms: str, objective_findings: str,
                 history: str, treatment_notes: str = "N/A"):
        self.findings = findings
        self.assessment_date = assessment_date
        self.subjective_symptoms = subjective_symptoms
        self.objective_findings = objective_findings
        self.history = history
        self.treatment_notes = treatment_notes

    def to_dict(self):
        return {"findings": self.findings,
                "assessment_date": self.assessment_date,
                "subjective_symptoms": self.subjective_symptoms,
                "objective_findings": self.objective_findings,
                "history": self.history,
                "treatment_notes": self.treatment_notes}

class Reassessment:
    def __init__(self, reassessment_date: date,
                 progress_notes: str, objective_findings: str,
                 recommendations: str):
        self.reassessment_date = reassessment_date
        self.progress_notes = progress_notes
        self.objective_findings = objective_findings
        self.recommendations = recommendations

    def to_dict(self):
        return {"reassessment_date": self.reassessment_date,
                "progress_notes": self.progress_notes,
                "objective_findings": self.objective_findings,
                "recommendations": self.recommendations}

class Patient:
    def __init__(self, name: str, contact: str, dob: date, gender: str,
                 medical_aid: str, medical_aid_number: str,
                 occupation: str, referral_diagnosis: str,
                 physiotherapist_name: str,
                 medical_history: MedicalHistory,
                 diagnosis: Diagnosis,
                 treatment_plan: TreatmentPlan):
        self.name = name
        self.contact = contact
        self.dob = dob
        self.gender = gender
        self.medical_aid = medical_aid
        self.medical_aid_number = medical_aid_number
        self.occupation = occupation
        self.referral_diagnosis = referral_diagnosis
        self.physiotherapist_name = physiotherapist_name
        self.medical_history = medical_history
        self.diagnosis = diagnosis
        self.treatment_plan = treatment_plan
        self.assessments = []
        self.reassessments = []

    def add_assessment(self, assessment: Assessment):
        self.assessments.append(assessment)

    def add_reassessment(self, reassessment: Reassessment):
        self.reassessments.append(reassessment)

    def to_dict(self):
        return {"name": self.name,
                "contact": self.contact,
                "dob": self.dob,
                "gender": self.gender,
                "medical_aid": self.medical_aid,
                "medical_aid_number": self.medical_aid_number,
                "occupation": self.occupation,
                "referral_diagnosis": self.referral_diagnosis,
                "physiotherapist_name": self.physiotherapist_name,
                "medical_history": self.medical_history.to_dict(),
                "diagnosis": self.diagnosis.to_dict(),
                "treatment_plan": self.treatment_plan.to_dict(),
                "assessments": [a.to_dict() for a in self.assessments],
                "reassessments": [r.to_dict() for r in self.reassessments]}

    @classmethod
    def from_dict(cls, data):
        mh = MedicalHistory(**data["medical_history"])
        diag = Diagnosis(**data["diagnosis"])
        tp_data = data["treatment_plan"]
        tp = TreatmentPlan(tp_data.get("exercises", []),
                           tp_data.get("therapies", []),
                           tp_data.get("modalities", []),
                           tp_data.get("interventions", ""),
                           tp_data.get("home_exercise_program", ""))
        patient = cls(data["name"], data["contact"], data["dob"],
                      data["gender"], data["medical_aid"],
                      data["medical_aid_number"], data["occupation"],
                      data["referral_diagnosis"], data["physiotherapist_name"],
                      mh, diag, tp)
        for a in data.get("assessments", []):
            patient.assessments.append(
                Assessment(a.get("findings", ""),
                           a.get("assessment_date", date.today()),
                           a.get("subjective_symptoms", ""),
                           a.get("objective_findings", ""),
                           a.get("history", ""),
                           a.get("treatment_notes", "N/A"))
            )
        for r in data.get("reassessments", []):
            patient.reassessments.append(
                Reassessment(r.get("reassessment_date", date.today()),
                             r.get("progress_notes", ""),
                             r.get("objective_findings", ""),
                             r.get("recommendations", ""))
            )
        return patient

# Main Application
class PhysioApp:
    def __init__(self, root):
        self.root = root

        # ─── Load ICD-10 lookup into memory ────────────────────────────
        with open("icd10.json", "r", encoding="utf-8") as f:
            # expect a dict: { "A00": "Cholera", … }
            self.icd_data = json.load(f)
        # ─────────────────────────────────────────────────────────────────

        self.root.title("Physiotherapy Patient Management System")
        self.root.geometry("800x600")
        self.patients = {}
        self.patient_counter = 1
        self.current_patient = None
        self.frames = {}
        self.patient_var = tk.StringVar()

        self.container = ttk.Frame(self.root)
        self.container.pack(fill="both", expand=True)

        self.create_main_menu()
        self.load_patients()

    def suggest_icd(self, query):
        """
        Return up to 10 (code, description) tuples from self.icd_data
        where the query appears (case-insensitive) in either code or desc.
        """
        q = query.strip().lower()
        if not q:
            return []
        matches = []
        for code, desc in self.icd_data.items():
            if q in code.lower() or q in desc.lower():
                matches.append((code, desc))
                if len(matches) >= 10:
                    break
        return matches

    def format_dob(self, event):
        s = re.sub(r"[^\d]", "", self.dob_entry.get())
        parts = []
        if len(s) >= 4:
            parts.append(s[:4])
            if len(s) >= 6:
                parts.append(s[4:6])
                parts.append(s[6:8])
            else:
                parts.append(s[4:])
        else:
            parts.append(s)
        new = '-'.join(parts)
        self.dob_entry.delete(0, tk.END)
        self.dob_entry.insert(0, new[:10])

    def validate_contact(self, event):
        s = re.sub(r"\D", "", self.contact_entry.get())[:10]
        self.contact_entry.delete(0, tk.END)
        self.contact_entry.insert(0, s)

    def _bind_mousewheel(self, canvas):
        canvas.bind_all("<MouseWheel>", lambda e: canvas.yview_scroll(int(-1*(e.delta/120)), "units"))

    def create_main_menu(self):
        self.clear_container()
        frame = ttk.Frame(self.container)
        frame.pack(fill="both", expand=True)
        tk.Label(frame, text="Physiotherapy Patient Management System", font=("Arial", 16)).pack(pady=20)
        tk.Button(frame, text="Patient Management", command=self.create_patient_frame).pack(pady=10)
        tk.Button(frame, text="Assessment & Reassessment", command=self.create_assessment_frame).pack(pady=10)
        tk.Button(frame, text="Generate Report", command=self.create_report_frame).pack(pady=10)
        tk.Button(frame, text="Exit", command=self.root.quit).pack(pady=10)

    def clear_container(self):
        for w in self.container.winfo_children(): w.destroy()
        self.frames.clear()

    def create_patient_frame(self):
        self.clear_container()
        canvas = tk.Canvas(self.container)
        scrollbar = ttk.Scrollbar(self.container, orient="vertical", command=canvas.yview)
        scrollable = ttk.Frame(canvas)
        scrollable.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
        canvas.create_window((0,0), window=scrollable, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        self.frames['patient'] = scrollable
        self._bind_mousewheel(canvas)

        fields = [
            ("Name", tk.Entry), ("Contact", tk.Entry), ("DOB (YYYY-MM-DD)", tk.Entry),
            ("Gender", tk.Entry), ("Medical Aid", tk.Entry), ("Medical Aid Number", tk.Entry),
            ("Occupation", tk.Entry), ("Referral Diagnosis", tk.Entry), ("Physiotherapist Name", tk.Entry)
        ]
        self.patient_entries = {}
        for i, (label, widget) in enumerate(fields):
            tk.Label(scrollable, text=label + ":").grid(row=i, column=0, padx=5, pady=5, sticky='w')
            entry = widget(scrollable)
            entry.grid(row=i, column=1, padx=5, pady=5, sticky='ew')
            self.patient_entries[label] = entry
            if label == 'Contact':
                self.contact_entry = entry
                entry.bind('<KeyRelease>', self.validate_contact)
            if label.startswith('DOB'):
                self.dob_entry = entry
                entry.bind('<KeyRelease>', self.format_dob)

        # Medical History
        offset = len(fields)
        tk.Label(scrollable, text="Medical History").grid(row=offset, column=0, columnspan=2, pady=10)
        tk.Label(scrollable, text="Conditions (comma-separated):").grid(row=offset+1, column=0, sticky='w')
        self.conditions_entry = tk.Entry(scrollable)
        self.conditions_entry.grid(row=offset+1, column=1, sticky='ew')
        tk.Label(scrollable, text="Surgeries (comma-separated):").grid(row=offset+2, column=0, sticky='w')
        self.surgeries_entry = tk.Entry(scrollable)
        self.surgeries_entry.grid(row=offset+2, column=1, sticky='ew')
        tk.Label(scrollable, text="Allergies (comma-separated):").grid(row=offset+3, column=0, sticky='w')
        self.allergies_entry = tk.Entry(scrollable)
        self.allergies_entry.grid(row=offset+3, column=1, sticky='ew')

        # Diagnosis
        diag_off = offset + 4
        tk.Label(scrollable, text="Diagnosis").grid(row=diag_off, column=0, columnspan=2, pady=10)
        tk.Label(scrollable, text="Diagnosis:").grid(row=diag_off+1, column=0, sticky='w')
        self.diagnosis_entry = tk.Entry(scrollable)
        self.diagnosis_entry.grid(row=diag_off+1, column=1, sticky='ew')
        tk.Label(scrollable, text="ICD Code:").grid(row=diag_off+2, column=0, sticky='w')
        self.icd_code_entry = tk.Entry(scrollable)
        self.icd_code_entry.grid(row=diag_off+2, column=1, sticky='ew')

        # --- ICD-10 suggestion dropdown (step 3) ---
        self.icd_suggestions = tk.Listbox(scrollable, height=6)
        self.icd_suggestions.grid(row=diag_off+3, column=1, padx=5, sticky='ew')
        self.icd_suggestions.bind("<<ListboxSelect>>", self.on_icd_select)
        self.icd_suggestions.lower()   # hide it until there are matches
        # bind typing event on the diagnosis field
        self.diagnosis_entry.bind("<KeyRelease>", self.on_diagnosis_type)

        # Treatment Plan
        tp_off = diag_off + 4
        tk.Label(scrollable, text="Treatment Plan").grid(row=tp_off, column=0, columnspan=2, pady=10)
        tk.Label(scrollable, text="Exercises (comma-separated):").grid(row=tp_off+1, column=0, sticky='w')
        self.exercises_entry = tk.Entry(scrollable)
        self.exercises_entry.grid(row=tp_off+1, column=1, sticky='ew')
        tk.Label(scrollable, text="Therapies (comma-separated):").grid(row=tp_off+2, column=0, sticky='w')
        self.therapies_entry = tk.Entry(scrollable)
        self.therapies_entry.grid(row=tp_off+2, column=1, sticky='ew')
        tk.Label(scrollable, text="Modalities (comma-separated):").grid(row=tp_off+3, column=0, sticky='w')
        self.modalities_entry = tk.Entry(scrollable)
        self.modalities_entry.grid(row=tp_off+3, column=1, sticky='ew')
        tk.Label(scrollable, text="Interventions:").grid(row=tp_off+4, column=0, sticky='w')
        self.interventions_entry = tk.Entry(scrollable)
        self.interventions_entry.grid(row=tp_off+4, column=1, sticky='ew')
        tk.Label(scrollable, text="Home Exercise Program:").grid(row=tp_off+5, column=0, sticky='w')
        self.home_exercise_entry = tk.Entry(scrollable)
        self.home_exercise_entry.grid(row=tp_off+5, column=1, sticky='ew')

        tk.Button(scrollable, text="Add Patient", command=self.add_patient).grid(row=tp_off+6, column=0, columnspan=2, pady=10)
        tk.Button(scrollable, text="Back to Menu", command=self.create_main_menu).grid(row=tp_off+7, column=0, columnspan=2, pady=20)
        scrollable.columnconfigure(1, weight=1)

    def add_patient(self):
        name = self.patient_entries['Name'].get().strip()
        contact = self.contact_entry.get().strip()
        dob_str = self.dob_entry.get().strip()
        # validate contact
        if not re.fullmatch(r"\d{10}", contact):
            return messagebox.showerror("Error", "Contact must be exactly 10 digits.")
        # validate dob
        try:
            dob = datetime.strptime(dob_str, "%Y-%m-%d").date()
        except ValueError:
            return messagebox.showerror("Error", "DOB must be in YYYY-MM-DD format.")
        if dob < date(1900,1,1) or dob > date.today():
            return messagebox.showerror("Error", "DOB must be between 1900-01-01 and today.")
        gender = self.patient_entries['Gender'].get().strip()
        medical_aid = self.patient_entries['Medical Aid'].get().strip()
        medical_aid_number = self.patient_entries['Medical Aid Number'].get().strip()
        occupation = self.patient_entries['Occupation'].get().strip()
        referral_diagnosis = self.patient_entries['Referral Diagnosis'].get().strip()
        physio_name = self.patient_entries['Physiotherapist Name'].get().strip()
        if not (name and referral_diagnosis and physio_name):
            return messagebox.showerror("Error", "Name, Referral Diagnosis, and Physiotherapist Name are required.")
        conditions = [c.strip() for c in self.conditions_entry.get().split(',') if c.strip()] or ['none']
        surgeries = [s.strip() for s in self.surgeries_entry.get().split(',') if s.strip()] or ['none']
        allergies = [a.strip() for a in self.allergies_entry.get().split(',') if a.strip()] or ['none']
        diagnosis_txt = self.diagnosis_entry.get().strip() or 'N/A'
        icd = self.icd_code_entry.get().strip() or 'N/A'
        exercises = [e.strip() for e in self.exercises_entry.get().split(',') if e.strip()] or ['none']
        therapies = [t.strip() for t in self.therapies_entry.get().split(',') if t.strip()] or ['none']
        modalities = [m.strip() for m in self.modalities_entry.get().split(',') if m.strip()] or ['none']
        interventions = self.interventions_entry.get().strip() or 'Initial treatment'
        home_prog = self.home_exercise_entry.get().strip() or 'Home exercises prescribed'

        mh = MedicalHistory(conditions, surgeries, allergies)
        diag = Diagnosis(diagnosis_txt, icd)
        tp = TreatmentPlan(exercises, therapies, modalities, interventions, home_prog)
        patient = Patient(name, contact, dob, gender, medical_aid, medical_aid_number,
                          occupation, referral_diagnosis, physio_name, mh, diag, tp)
        pid = f"patient_{self.patient_counter:03d}"
        self.patients[pid] = patient
        self.patient_counter += 1
        self.current_patient = patient
        self.save_patients()
        messagebox.showinfo("Success", f"Patient {name} added!")

    def create_assessment_frame(self):
        self.clear_container()
        canvas = tk.Canvas(self.container)
        scrollbar = ttk.Scrollbar(self.container, orient="vertical", command=canvas.yview)
        scrollable = ttk.Frame(canvas)
        scrollable.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
        canvas.create_window((0,0), window=scrollable, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        self.frames['assessment'] = scrollable
        self._bind_mousewheel(canvas)

        tk.Label(scrollable, text="Select Patient:").grid(row=0, column=0, pady=5)
        self.assess_menu = ttk.OptionMenu(scrollable, self.patient_var, *self.get_patient_names(), command=self.select_patient)
        self.assess_menu.grid(row=0, column=1, pady=5, sticky='ew')

        labels = ["Assessment Findings:", "Subjective Symptoms:", "Objective Findings:", "History:", "Treatment Notes:"]
        self.assess_entries = {}
        for i, text in enumerate(labels, start=1):
            tk.Label(scrollable, text=text).grid(row=i, column=0, sticky='nw', padx=5, pady=5)
            txt = tk.Text(scrollable, height=3 if i<5 else 5, width=40)
            txt.grid(row=i, column=1, padx=5, pady=5, sticky='ew')
            self.assess_entries[text] = txt

        tk.Button(scrollable, text="Add Assessment", command=self.add_assessment).grid(row=6, column=0, columnspan=2, pady=10)
        tk.Button(scrollable, text="Add Reassessment", command=self.add_reassessment).grid(row=7, column=0, columnspan=2, pady=10)
        tk.Button(scrollable, text="Back to Menu", command=self.create_main_menu).grid(row=8, column=0, columnspan=2, pady=20)
        scrollable.columnconfigure(1, weight=1)

    def add_assessment(self):
        if not self.current_patient:
            return messagebox.showerror("Error", "No patient selected.")
        find = self.assess_entries["Assessment Findings:"].get('1.0', tk.END).strip()
        subj = self.assess_entries["Subjective Symptoms:"].get('1.0', tk.END).strip()
        obj = self.assess_entries["Objective Findings:"].get('1.0', tk.END).strip()
        hist = self.assess_entries["History:"].get('1.0', tk.END).strip() or "N/A"
        notes = self.assess_entries["Treatment Notes:"].get('1.0', tk.END).strip() or "N/A"
        if not (find and subj and obj):
            return messagebox.showerror("Error", "Please fill Findings, Subjective, and Objective.")
        a = Assessment(find, date.today(), subj, obj, hist, notes)
        self.current_patient.add_assessment(a)
        self.save_patients()
        messagebox.showinfo("Success", "Assessment added.")

    def add_reassessment(self):
        if not self.current_patient:
            return messagebox.showerror("Error", "No patient selected.")
        prog = self.assess_entries["Treatment Notes:"].get('1.0', tk.END).strip()
        obj = self.assess_entries["Objective Findings:"].get('1.0', tk.END).strip()
        rec = hist = self.assess_entries["History:"].get('1.0', tk.END).strip() or "N/A"
        if not (prog and obj):
            return messagebox.showerror("Error", "Please fill progress and objective.")
        r = Reassessment(date.today(), prog, obj, rec)
        self.current_patient.add_reassessment(r)
        self.save_patients()
        messagebox.showinfo("Success", "Reassessment added.")

    def create_report_frame(self):
        self.clear_container()
        canvas = tk.Canvas(self.container)
        scrollbar = ttk.Scrollbar(self.container, orient="vertical", command=canvas.yview)
        scrollable = ttk.Frame(canvas)
        scrollable.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
        canvas.create_window((0,0), window=scrollable, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        self.frames['report'] = scrollable
        self._bind_mousewheel(canvas)

        tk.Label(scrollable, text="Select Patient for Report:").grid(row=0, column=0, pady=5)
        self.report_menu = ttk.OptionMenu(scrollable, self.patient_var, *self.get_patient_names(), command=self.select_patient)
        self.report_menu.grid(row=0, column=1, pady=5, sticky='ew')
        tk.Button(scrollable, text="Generate Word Report", command=self.generate_word_report).grid(row=1, column=0, columnspan=2, pady=10)
        tk.Button(scrollable, text="Back to Menu", command=self.create_main_menu).grid(row=2, column=0, columnspan=2, pady=20)
        scrollable.columnconfigure(1, weight=1)

    def generate_word_report(self):
        if not self.current_patient:
            return messagebox.showerror("Error", "No patient selected.")
        doc = Document()
        doc.add_heading(f"Physio Report: {self.current_patient.name}", level=0)
        doc.add_paragraph(f"DOB: {self.current_patient.dob}")
        doc.add_paragraph(f"Contact: {self.current_patient.contact}")
        doc.add_paragraph(f"Med Aid: {self.current_patient.medical_aid} #{self.current_patient.medical_aid_number}")
        doc.add_paragraph(f"Referral Dx: {self.current_patient.referral_diagnosis}")
        if self.current_patient.assessments:
            a = self.current_patient.assessments[-1]
            doc.add_heading("Assessment", level=1)
            doc.add_paragraph(f"{a.subjective_symptoms}. Objective: {a.objective_findings}. History: {a.history}.")
        doc.add_heading("Treatment Plan", level=1)
        doc.add_paragraph(self.current_patient.treatment_plan.interventions +
                          ". Home program: " + self.current_patient.treatment_plan.home_exercise_program)
        if self.current_patient.reassessments:
            r = self.current_patient.reassessments[-1]
            doc.add_heading("Reassessment", level=1)
            doc.add_paragraph(f"Progress: {r.progress_notes}. Objective: {r.objective_findings}. Rec: {r.recommendations}.")
        doc.add_paragraph("Kind regards,\n" + self.current_patient.physiotherapist_name)
        fn = f"{self.current_patient.name.replace(' ','_')}_report.docx"
        try:
            doc.save(fn)
            messagebox.showinfo("Success", f"Report saved as {fn}")
        except Exception as e:
            messagebox.showerror("Error", f"Unable to save report: {e}")
    # --- step 4: ICD lookup callbacks ---
    def on_diagnosis_type(self, event):
        text = self.diagnosis_entry.get()
        matches = self.suggest_icd(text)
        lb = self.icd_suggestions
        lb.delete(0, tk.END)
        if not matches:
            lb.lower()
            return
        for e in matches:
            lb.insert(tk.END, f"{e['code']}: {e['description']}")
        lb.lift()

    def on_icd_select(self, event):
        sel = event.widget.curselection()
        if not sel:
            return
        line = event.widget.get(sel[0])
        code, desc = line.split(":", 1)
        # fill both entries
        self.icd_code_entry.delete(0, tk.END)
        self.icd_code_entry.insert(0, code.strip())
        self.diagnosis_entry.delete(0, tk.END)
        self.diagnosis_entry.insert(0, desc.strip())
        # hide suggestion list
        event.widget.delete(0, tk.END)
        event.widget.lower()
    
    def save_patients(self):
        try:
            data = {pid: p.to_dict() for pid,p in self.patients.items()}
            with open("patients.json","w") as f:
                json.dump(data,f,default=str)
        except Exception as e:
            messagebox.showerror("Error", f"Save failed: {e}")

    def load_patients(self):
        try:
            with open("patients.json","r") as f:
                data = json.load(f, object_hook=str_to_date)
            for pid,d in data.items():
                self.patients[pid] = Patient.from_dict(d)
            self.patient_counter = len(self.patients)+1
            if self.patients:
                self.current_patient = list(self.patients.values())[-1]
            self.update_menus()
        except FileNotFoundError:
            pass
        except Exception as e:
            messagebox.showerror("Error", f"Load failed: {e}")

    def get_patient_names(self):
        names = [p.name for p in self.patients.values()]
        return names if names else ["No patients"]

    def select_patient(self, *args):
        name = self.patient_var.get()
        self.current_patient = next((p for p in self.patients.values() if p.name==name), None)

    def update_menus(self):
        for key in ('assessment','report'):
            if key in self.frames:
                menu = self.frames[key].nametowidget(self.frames[key].winfo_children()[1])
                menu['menu'].delete(0,'end')
                for n in self.get_patient_names():
                    menu['menu'].add_command(label=n, command=tk._setit(self.patient_var,n))

if __name__ == '__main__':
    root = tk.Tk()
    app = PhysioApp(root)
    root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\andil\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\andil\AppData\Local\Temp\ipykernel_24520\3684542029.py", line 539, in on_diagnosis_type
    matches = self.suggest_icd(text)
              ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\andil\AppData\Local\Temp\ipykernel_24520\3684542029.py", line 256, in suggest_icd
    for code, desc in self.icd_data.items():
                      ^^^^^^^^^^^^^^^^^^^
AttributeError: 'list' object has no attribute 'items'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\andil\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\andil\AppData\Local\Temp\ipykernel_24520\3684542029.py", line 539, in on_diagnosis_type
    matches = self.suggest_icd(text)
              ^^^^^^^^^^^^^^^^^^